## 1. Concessão de cartão de crédito
<p>Os bancos recebem <em>muitos</em> pedidos de cartões de crédito. Analisar manualmente essas solicitações é operacionalmente inviável, sujeito a erros e demorado. Felizmente, essa tarefa pode ser automatizada com o poder do <a href="https://medium.com/camilawaltrick/introducao-machine-learning-o-que-e-tipos-de-aprendizado-de-maquina-445dcfb708f0">aprendizado de máquina</a> e praticamente todos os bancos fazem isso.
Neste notebook, estarei construindo um modelo preditor para concessão de  cartão de crédito. Todas as etapas estão bem descritas, em formato de tutorial.</p>

<p>Será utilizado o <a href="http://archive.ics.uci.edu/ml/datasets/credit+approval">conjunto de dados Credit Card Approval</a> do repositório UCI Machine Learning. A estrutura deste notebook é a seguinte:</p>
<ul>
<li>Primeiro, começaremos carregando e visualizando o conjunto de dados.</li>
<li>Veremos que o conjunto de dados tem uma mistura de recursos numéricos e não numéricos, que contém valores de diferentes intervalos, além de várias entradas ausentes.</li>
<li>Teremos que pré-processar o conjunto de dados para garantir que o modelo de aprendizado de máquina que escolhemos possa fazer boas previsões.</li>
<li>Por fim, criaremos um modelo de aprendizado de máquina que pode prever se a solicitação de cartão de crédito de um indivíduo será aceita.</li>
</ul>

In [ ]:
# Bibliotecas
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV

In [ ]:
# Carregar e checar o dataset
cc_apr = pd.read_csv("/content/crx.data", header=None)
cc_apr.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,00202,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,00043,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,00280,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,00100,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,00120,0,+


**Ao  carregar e visualizar o conjunto de dados, vemos que os dados são confidenciais, pois os nomes das variáveis foram anonimizados.**

## 2. Conferindo os dados
<p>A ausência de rótulos descritivos para as colunas pode parecer um pouco confuso à primeira vista, mas pesquisando sobre esse dataset, foi encontrado <a href="http://rstudio-pubs-static.s3.amazonaws.com/73039_9946de135c0a49daa7a0a9eda4a67a72.html">este blog</a> que dá uma boa visão geral dos possíveis rótulos. Seguindo a ordem que se apresentam, temos as colunas <code>Gender</code>, <code>Age</code>, <code>Debt</code>, <code>Married</code>, <code>BankCustomer</code>, <code>EducationLevel</code>, <code>Ethnicity</code>, <code>YearsEmployed</code>, <code>PriorDefault</code>, <code>Employed</code>, <code>CreditScore</code>, <code>DriversLicense</code>, <code>Citizen</code>, <code>ZipCode</code>, <code>Income</code> e <code>ApprovalStatus</code>. Isso nos dá um bom ponto de partida e podemos mapear esses recursos em relação às colunas na saída. </p>
<p>Como podemos ver em nossa primeira olhada nos dados, o conjunto de dados tem uma mistura de variáveis numéricas e não numéricas. Isso pode ser corrigido com algum pré-processamento, mas antes de fazermos isso, vamos aprender um pouco mais sobre o conjunto de dados para ver se há outros problemas de conjunto de dados que precisam ser corrigidos.</p>

In [ ]:
# Resumo estatístico
cc_apr_desc = cc_apr.describe()
print(cc_apr_desc)

print('\n')

# Infos do conjuntod e dados
cc_apr_info = cc_apr.info()
print(cc_apr_info)

print('\n')

# Inspecionar valores ausentes no conjunto de dados
cc_apr.tail(17)

               2           7          10             14
count  690.000000  690.000000  690.00000     690.000000
mean     4.758725    2.223406    2.40000    1017.385507
std      4.978163    3.346513    4.86294    5210.102598
min      0.000000    0.000000    0.00000       0.000000
25%      1.000000    0.165000    0.00000       0.000000
50%      2.750000    1.000000    0.00000       5.000000
75%      7.207500    2.625000    3.00000     395.500000
max     28.000000   28.500000   67.00000  100000.000000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       690 non-null    object 
 1   1       690 non-null    object 
 2   2       690 non-null    float64
 3   3       690 non-null    object 
 4   4       690 non-null    object 
 5   5       690 non-null    object 
 6   6       690 non-null    object 
 7   7       690 non-null    float64
 8   8       690 no

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
673,?,29.50,2.000,y,p,e,h,2.000,f,f,0,f,g,00256,17,-
674,a,37.33,2.500,u,g,i,h,0.210,f,f,0,f,g,00260,246,-
675,a,41.58,1.040,u,g,aa,v,0.665,f,f,0,f,g,00240,237,-
676,a,30.58,10.665,u,g,q,h,0.085,f,t,12,t,g,00129,3,-
677,b,19.42,7.250,u,g,m,v,0.040,f,t,1,f,g,00100,1,-
678,a,17.92,10.210,u,g,ff,ff,0.000,f,f,0,f,g,00000,50,-
679,a,20.08,1.250,u,g,c,v,0.000,f,f,0,f,g,00000,0,-
680,b,19.50,0.290,u,g,k,v,0.290,f,f,0,f,g,00280,364,-
681,b,27.83,1.000,y,p,d,h,3.000,f,f,0,f,g,00176,537,-
682,b,17.08,3.290,u,g,i,v,0.335,f,f,0,t,g,00140,2,-


## 3. Dividindo o conjunto de dados em conjuntos de treinamento e teste
<p>Agora, dividiremos nossos dados em conjunto de treinamento e conjunto de teste para preparar nossos dados para duas fases diferentes da modelagem de aprendizado de máquina: treinamento e teste. Idealmente, nenhuma informação dos dados de teste deve ser usada para pré-processar os dados de treinamento ou deve ser usada para direcionar o processo de treinamento de um modelo de aprendizado de máquina. Portanto, primeiro dividimos os dados e depois os pré-processamos.</p>
<p>Além disso, recursos como <code>DriversLicense</code> e <code>ZipCode</code> não são tão importantes quanto os outros recursos no conjunto de dados para prever aprovações de cartão de crédito. Para ter uma noção melhor, podemos medir sua <a href="https://realpython.com/numpy-scipy-pandas-correlation-python/">correlação estatística</a> com os rótulos do conjunto de dados. Mas isso está fora do escopo deste projeto. Devemos excluí-los para projetar nosso modelo de aprendizado de máquina com o melhor conjunto de recursos. Na literatura de ciência de dados, isso costuma ser chamado de <em>seleção de recursos (feature engineering)</em>. </p>

In [ ]:
# Excluir variáveis 11 e 13
cc_apr = cc_apr.drop([11, 13], axis=1)

# Dividir em conjuntos de treinamento e teste
cc_apr_treino, cc_apr_teste = train_test_split(cc_apr, test_size=0.33, random_state=42)

## 4. Manipulando os valores ausentes (parte i)
<p>Agora que dividimos nossos dados, podemos lidar com alguns dos problemas que identificamos ao inspecionar o DataFrame, incluindo:</p>
<ul>
<li>Nosso conjunto de dados contém dados numéricos e não numéricos (especificamente dados que são dos tipos <code>float64</code>, <code>int64</code> e <code>object</code>). Especificamente, os recursos 2, 7, 10 e 14 contêm valores numéricos (dos tipos float64, float64, int64 e int64, respectivamente) e todos os outros recursos contêm valores não numéricos.</li>
<li>O conjunto de dados também contém valores de vários intervalos. Alguns recursos têm um intervalo de valores de 0 a 28, alguns têm um intervalo de 2 a 67 e alguns têm um intervalo de 1017 a 100000. Além disso, podemos obter informações estatísticas úteis (como <code>mean</code> , <code>max</code> e <code>min</code>) sobre os recursos que possuem valores numéricos. </li>
<li>Finalmente, o conjunto de dados tem valores ausentes, dos quais cuidaremos nesta etapa. Os valores ausentes no conjunto de dados são rotulados com '?'.</li>
</ul>
<p>Agora, vamos substituir temporariamente esses pontos de interrogação de valores ausentes por NaN.</p>

In [ ]:
# Substituir os '?'s por NaN nos conjuntos de treinamento e teste
cc_apr_treino = cc_apr_treino.replace('?', np.NaN)
cc_apr_teste = cc_apr_teste.replace('?', np.NaN)

## 5. Manipulando os valores ausentes (parte ii)
<p>Substituímos todos os pontos de interrogação por NaNs. Isso vai nos ajudar no próximo tratamento de valor ausente que vamos realizar.</p>
<p>Uma questão importante levantada aqui é <em>por que estamos dando tanta importância aos valores ausentes</em>? Eles não podem ser simplesmente ignorados? Ignorar os valores ausentes pode afetar fortemente o desempenho de um modelo de aprendizado de máquina. Ao ignorar os valores ausentes, nosso modelo de aprendizado de máquina pode perder informações sobre o conjunto de dados que podem ser úteis para seu treinamento. Então, existem muitos modelos que não podem lidar com valores ausentes implicitamente, como a Análise Discriminante Linear (LDA). </p>
<p>Então, para evitar esse problema, vamos imputar os valores ausentes com uma estratégia chamada imputação média.</p>

In [ ]:
# Atribuir os valores ausentes com imputação média
cc_apr_treino.fillna(cc_apr_treino.mean(), inplace=True)
cc_apr_teste.fillna(cc_apr_treino.mean(), inplace=True)

# Contar o número de NaNs nos conjuntos de dados e imprimir as contagens para verificar
print(cc_apr_treino.isnull().sum())
print(cc_apr_teste.isnull().sum())

0     8
1     5
2     0
3     6
4     6
5     7
6     7
7     0
8     0
9     0
10    0
12    0
14    0
15    0
dtype: int64
0     4
1     7
2     0
3     0
4     0
5     2
6     2
7     0
8     0
9     0
10    0
12    0
14    0
15    0
dtype: int64


<ipython-input-24-6221ce57fdb7>:2: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  cc_apr_treino.fillna(cc_apr_treino.mean(), inplace=True)
<ipython-input-24-6221ce57fdb7>:3: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  cc_apr_teste.fillna(cc_apr_treino.mean(), inplace=True)


## 6. Manipulando os valores ausentes (parte iii)
<p>Resolvemos com sucesso os valores ausentes presentes nas colunas numéricas. Ainda faltam alguns valores a serem imputados para as colunas 0, 1, 3, 4, 5, 6 e 13. Todas essas colunas contêm dados não numéricos e é por isso que a estratégia de imputação média não funcionaria aqui. Isso precisa de um tratamento diferente. </p>
<p>Iremos imputar esses valores ausentes com os valores mais frequentes presentes nas respectivas colunas. Esta é uma <a href="https://www.datacamp.com/community/tutorials/categorical-data">boa prática</a> quando se trata de imputar valores ausentes para dados categóricos em geral.</p>

In [ ]:
# Contar o número de NaNs nos conjuntos de dados e imprimir as contagens para verificar

for col in cc_apr_treino.columns:
    # Verificar se a coluna é do tipo objeto
    if cc_apr_treino[col].dtypes == 'object':
        # Atribuir com o valor mais frequente
        cc_apr_treino = cc_apr_treino.fillna(cc_apr_treino[col].value_counts().index[0])
        cc_apr_teste = cc_apr_teste.fillna(cc_apr_treino[col].value_counts().index[0])

print(cc_apr_treino.isnull().sum())
print(cc_apr_teste.isnull().sum())

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
12    0
14    0
15    0
dtype: int64
0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
12    0
14    0
15    0
dtype: int64


## 7. Pré-processando os dados (parte i)
<p>Os valores ausentes agora estão tratados com sucesso.</p>
<p>Ainda é necessário um pré-processamento de dados menor, mas essencial, antes de prosseguirmos para a construção de nosso modelo de aprendizado de máquina. Vamos dividir essas etapas de pré-processamento restantes em duas tarefas principais:</p>
<ol>
<li>Converter os dados não numéricos em numéricos.</li>
<li>Padronizar os valores dos recursos para um intervalo uniforme.</li>
</ol>
<p>Primeiro, converteremos todos os valores não numéricos em numéricos. Fazemos isso porque não apenas resulta em uma computação mais rápida, mas também porque muitos modelos de aprendizado de máquina (como o XGBoost) - e especialmente os desenvolvidos usando o scikit-learn, exigem que os dados estejam em um formato estritamente numérico. Faremos isso usando o método <code>get_dummies()</code> dos pandas.</p>

In [ ]:
# Converter os recursos categóricos nos conjuntos de treinamento e teste de forma independente
cc_apr_treino = pd.get_dummies(cc_apr_treino)
cc_apr_teste = pd.get_dummies(cc_apr_teste)

# Reindexar as colunas do conjunto de teste alinhando com o conjunto de treinamento
cc_apr_teste = cc_apr_teste.reindex(columns=cc_apr_treino.columns, fill_value=0)

## 8. Pré-processamento dos dados (parte ii)
<p>Agora, resta apenas uma etapa final de pré-processamento de dimensionamento antes de podermos ajustar um modelo de aprendizado de máquina aos dados. </p>
<p>Agora, vamos tentar entender o que esses valores padronizados significam no mundo real. Vamos usar <code>CreditScore</code> como exemplo. A pontuação de crédito de uma pessoa é sua credibilidade com base em seu histórico de crédito. Quanto maior esse número, mais confiável financeiramente uma pessoa é considerada. Assim, um <code>CreditScore</code> de 1 é o mais alto, pois estamos redimensionando todos os valores para o intervalo de 0-1.</p>

In [ ]:
# Separar recursos e label em variáveis distintas
X_treino, y_treino = cc_apr_treino.iloc[:, :-1].values, cc_apr_treino.iloc[:, [-1]].values
X_teste, y_teste = cc_apr_teste.iloc[:, :-1].values, cc_apr_teste.iloc[:, [-1]].values

# Instanciar MinMaxScaler e redimensionar X_treino e X_teste
scaler = MinMaxScaler(feature_range=(0, 1))
rescaledX_treino = scaler.fit_transform(X_treino)
rescaledX_teste = scaler.transform(X_teste)

## 9. Ajustando um modelo de regressão logística ao conjunto de treino
<p>Essencialmente, prever se um pedido de cartão de crédito será aprovado ou não é uma tarefa de <a href="https://en.wikipedia.org/wiki/Statistical_classification">classificação</a>. De acordo com a UCI, nosso conjunto de dados contém mais instâncias que correspondem ao status "Negado" do que instâncias correspondentes ao status "Aprovado". Especificamente, de 690 instâncias, 383 (55,5%) solicitações foram negadas e 307 (44,5%) solicitações foram aprovadas. </p>
<p>Isso nos dá uma referência. Um bom modelo de aprendizado de máquina deve ser capaz de prever com precisão o status dos pedidos em relação a essas estatísticas.</p>
<p>Qual modelo devemos escolher? Uma pergunta a ser feita é: <em>os recursos que afetam o processo de decisão de aprovação de cartão de crédito estão correlacionados entre si?</em> Embora possamos medir a correlação, isso está fora do escopo deste notebook, portanto, vamos supor que eles realmente estão correlacionados por enquanto. Devido a essa correlação, aproveitaremos o fato de que os modelos lineares generalizados funcionam bem nesses casos. Vamos começar nossa modelagem de aprendizado de máquina com um modelo de regressão logística (um modelo linear generalizado).</p>

In [ ]:
# Instanciar um classificador LogisticRegression com valores de parâmetro padrão
logreg = LogisticRegression()

# Ajustar logreg ao conjunto de treino
logreg.fit(rescaledX_treino,y_treino)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression()

## 10. Fazendo previsões e avaliando o desempenho
<p>Mas qual é o desempenho do nosso modelo? </p>
<p>Vamos agora avaliar nosso modelo no conjunto de teste com relação à <a href="https://developers.google.com/machine-learning/crash-course/classification/accuracy">precisão de classificação</a> . Mas também vamos dar uma olhada na <a href="http://www.dataschool.io/simple-guide-to-confusion-matrix-terminology/">matriz de confusão</a> do modelo. No caso de prever pedidos de cartão de crédito, é importante ver se nosso modelo de aprendizado de máquina é igualmente capaz de prever status aprovado e negado, de acordo com a frequência desses rótulos em nosso conjunto de dados original. Se o nosso modelo não estiver tendo um bom desempenho nesse aspecto, ele pode acabar aprovando um pedido que deveria não ter sido aprovado. A matriz de confusão nos ajuda a visualizar o desempenho do nosso modelo, a partir desses aspectos. </p>

In [ ]:
# Usar logreg para prever instâncias do conjunto de teste e armazenar
y_pred = logreg.predict(rescaledX_teste)

# Obter a pontuação de precisão do modelo logreg e imprimir
print("Acurácia do classificador de regressão logística: ", logreg.score(rescaledX_teste,y_teste))

# Imprimir a matriz de confusão do modelo logreg
confusion_matrix(y_teste,y_pred)

Acurácia do classificador de regressão logística:  1.0


array([[103,   0],
       [  0, 125]])

## 11. Aplicando Grid Search e fazendo o modelo funcionar melhor
<p>Nosso modelo ficou muito bom! Na verdade, foi capaz de produzir uma pontuação de precisão de 100%.</p>
<p>Para a matriz de confusão, o primeiro elemento da primeira linha da matriz indica os verdadeiros negativos, significando o número de instâncias negativas (solicitaações negadas) previstas pelo modelo corretamente. E o último elemento da segunda linha da matriz indica os verdadeiros positivos, ou seja, o número de instâncias positivas (solicitações aprovadas) previstas pelo modelo corretamente.</p>
<p>Mas se não tivéssemos uma pontuação perfeita, o que fazer? Podemos realizar um <a href="https://machinelearningmastery.com/how-to-tune-algorithm-parameters-with-scikit-learn/">grid search</a> dos parâmetros do modelo para melhorar a capacidade do modelo em prever as solicitações de cartão de crédito.</p>
<p><a href="http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html">A implementação de regressão logística do scikit-learn</a> consiste em diferentes hiperparâmetros, mas vamos aplicar o Grid Search nos dois seguintes:</p>
<ul>
<li>tol</li>
<li>max_iter</li>
</ul>

In [ ]:
# Definir a grade de valores para tol e max_iter
tol = [0.01, 0.001 ,0.0001]
max_iter = [100, 150, 200]

# Criar um dicionário onde tol e max_iter são chaves e as listas de seus valores são os valores correspondentes
param_grid = dict(tol=tol, max_iter=max_iter)

## 12. Encontrar o modelo de melhor desempenho
<p>Definimos a grade de valores de hiperparâmetros e os convertemos em um único formato de dicionário que <code>GridSearchCV()</code> espera como um de seus parâmetros. Agora, iniciaremos o grid search para ver quais valores funcionam melhor.</p>
<p>Instanciaremos <code>GridSearchCV()</code> com nosso modelo <code>logreg</code> anterior com todos os dados que temos. Também instruiremos <code>GridSearchCV()</code> a realizar uma <a href="https://www.dataschool.io/machine-learning-with-scikit-learn/">validação cruzada</a > de cinco dobras.</p>
<p>Finalizaremos o notebook armazenando a melhor pontuação alcançada e os respectivos melhores parâmetros.</p>

In [ ]:
# Instanciar GridSearchCV com os parâmetros necessários
grid_model = GridSearchCV(estimator=logreg, param_grid=param_grid, cv=5)

# Ajustar grid_model aos dados
grid_model_result = grid_model.fit(rescaledX_treino, y_treino.ravel())

In [ ]:
# Resultados
best_score, best_params = grid_model_result.best_score_, grid_model_result.best_params_
print("Melhor score: %f com os parâmetros %s" % (best_score, best_params))

# Extrair o melhor modelo e avaliar no conjunto de teste
best_model = grid_model_result.best_estimator_
print("Acurácia do classificador de regressão logística: ", best_model.score(rescaledX_teste,y_teste))

Melhor score: 1.000000 com os parâmetros {'max_iter': 100, 'tol': 0.01}
Acurácia do classificador de regressão logística:  1.0


<p><strong>Revisando:</strong> Ao criar esse preditor de cartão de crédito, abordamos algumas das etapas de pré-processamento mais conhecidas, como dimensionamento, codificação de rótulo e imputação de valor ausente. Terminamos com um pouco de aprendizado de máquina para prever se o pedido de cartão de crédito de uma pessoa seria aprovado ou não receberia algumas informações sobre essa pessoa.</p>

<p><strong>Próximos passos:</strong> No mundo real, o desenvolvimento desse modelo baseado em regressão logística servia como um baseline para a construção de modelos mais robustos. Desta forma, como próximos passos, fica a criação de um pipeline com modelos (individuais ou ensemble) para avaliação de suas respectivas previsões.